# Notebook 3: Test và Sử dụng Mô hình
## Dự đoán từ tiếp theo - Next Word Prediction

Notebook này thực hiện:
1. Load các mô hình đã lưu (N-gram và LSTM)
2. Load vocabulary và các file cần thiết
3. Tạo hàm predict_next_word() như yêu cầu
4. Test với các ví dụ cụ thể
5. So sánh hiệu suất giữa 2 mô hình

## 1. Import thư viện

In [ ]:
import pickle
import numpy as np
from underthesea import word_tokenize
import os

print("Thư viện đã được import!")

## 2. Load Vocabulary và Data

In [ ]:
# Load vocabulary
print("Đang load vocabulary...")
with open('../data/processed/vocabulary.pkl', 'rb') as f:
    vocab_data = pickle.load(f)
    vocab = vocab_data['vocab']
    word2idx = vocab_data['word2idx']
    idx2word = vocab_data['idx2word']
    vocab_size = vocab_data['vocab_size']

print(f"✓ Vocabulary loaded: {vocab_size} words")

# Load config
with open('../data/processed/config.pkl', 'rb') as f:
    config = pickle.load(f)
    max_seq_len = config['max_seq_len']

print(f"✓ Config loaded: max_seq_len = {max_seq_len}")

## 3. Load Models

Cần import lại các class definition từ notebook 2

In [ ]:
# Import lại class definitions
from collections import defaultdict, Counter

# ============= N-gram Model Class =============
class NgramModel:
    """
    Mô hình N-gram được xây dựng từ đầu
    """
    def __init__(self, n=3, smoothing=0.01):
        self.n = n
        self.smoothing = smoothing
        self.ngram_counts = defaultdict(lambda: defaultdict(int))
        self.context_counts = defaultdict(int)
        self.vocab = set()
    
    def get_probabilities(self, context):
        context = tuple(context)
        context_count = self.context_counts[context]
        probabilities = {}
        vocab_size = len(self.vocab)
        
        if context_count > 0:
            for word in self.vocab:
                word_count = self.ngram_counts[context].get(word, 0)
                prob = (word_count + self.smoothing) / (context_count + self.smoothing * vocab_size)
                probabilities[word] = prob
        else:
            uniform_prob = 1.0 / vocab_size
            for word in self.vocab:
                probabilities[word] = uniform_prob
        
        return probabilities
    
    def predict_top_k(self, context, k=5):
        if len(context) >= self.n - 1:
            context = context[-(self.n-1):]
        
        probs = self.get_probabilities(context)
        sorted_words = sorted(probs.items(), key=lambda x: x[1], reverse=True)
        
        return [(word, prob) for word, prob in sorted_words[:k]]
    
    @staticmethod
    def load(filepath):
        with open(filepath, 'rb') as f:
            model_data = pickle.load(f)
        
        model = NgramModel(n=model_data['n'], smoothing=model_data['smoothing'])
        model.ngram_counts = defaultdict(lambda: defaultdict(int), model_data['ngram_counts'])
        model.context_counts = defaultdict(int, model_data['context_counts'])
        model.vocab = set(model_data['vocab'])
        
        return model

print("✓ NgramModel class defined")

In [ ]:
# ============= LSTM Model Class =============
def sigmoid(x):
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

def tanh(x):
    return np.tanh(x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

class SimpleLSTM:
    """
    LSTM đơn giản được xây dựng từ đầu bằng NumPy
    """
    def __init__(self, vocab_size, embedding_dim=50, hidden_dim=128, max_seq_len=10):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.max_seq_len = max_seq_len
    
    def forward_step(self, x_t, h_prev, c_prev):
        combined = np.concatenate([x_t, h_prev], axis=1)
        
        ft = sigmoid(np.dot(combined, self.Wf) + self.bf)
        it = sigmoid(np.dot(combined, self.Wi) + self.bi)
        c_tilde = tanh(np.dot(combined, self.Wc) + self.bc)
        c_t = ft * c_prev + it * c_tilde
        ot = sigmoid(np.dot(combined, self.Wo) + self.bo)
        h_t = ot * tanh(c_t)
        
        return h_t, c_t
    
    def forward(self, X):
        batch_size = X.shape[0]
        seq_len = X.shape[1]
        
        h = np.zeros((batch_size, self.hidden_dim))
        c = np.zeros((batch_size, self.hidden_dim))
        
        for t in range(seq_len):
            x_t = self.embedding[X[:, t]]
            h, c = self.forward_step(x_t, h, c)
        
        logits = np.dot(h, self.Wy) + self.by
        probs = softmax(logits)
        
        return probs, h
    
    def predict_top_k(self, X, k=5):
        probs, _ = self.forward(X)
        
        top_k_indices = np.argsort(probs, axis=1)[:, -k:][:, ::-1]
        top_k_probs = np.take_along_axis(probs, top_k_indices, axis=1)
        
        return top_k_indices, top_k_probs
    
    @staticmethod
    def load(filepath):
        with open(filepath, 'rb') as f:
            model_data = pickle.load(f)
        
        model = SimpleLSTM(
            vocab_size=model_data['vocab_size'],
            embedding_dim=model_data['embedding_dim'],
            hidden_dim=model_data['hidden_dim'],
            max_seq_len=model_data['max_seq_len']
        )
        
        model.embedding = model_data['embedding']
        model.Wf = model_data['Wf']
        model.bf = model_data['bf']
        model.Wi = model_data['Wi']
        model.bi = model_data['bi']
        model.Wc = model_data['Wc']
        model.bc = model_data['bc']
        model.Wo = model_data['Wo']
        model.bo = model_data['bo']
        model.Wy = model_data['Wy']
        model.by = model_data['by']
        
        return model

print("✓ SimpleLSTM class defined")

In [ ]:
# Load N-gram model
print("\nĐang load N-gram model...")
ngram_model = NgramModel.load('../models/ngram_model.pkl')
print(f"✓ N-gram model loaded (n={ngram_model.n})")

# Load LSTM model
print("\nĐang load LSTM model...")
lstm_model = SimpleLSTM.load('../models/lstm_model.pkl')
print(f"✓ LSTM model loaded")
print(f"  Vocab size: {lstm_model.vocab_size}")
print(f"  Embedding dim: {lstm_model.embedding_dim}")
print(f"  Hidden dim: {lstm_model.hidden_dim}")

## 4. Xây dựng hàm predict_next_word()

In [ ]:
def preprocess_input(text):
    """
    Tiền xử lý input text giống như trong training
    """
    import re
    
    # Chuyển về chữ thường
    text = text.lower().strip()
    
    # Loại bỏ ký tự đặc biệt không cần thiết
    text = re.sub(r'[^a-záàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệíìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữựýỳỷỹỵđ0-9\s]', '', text)
    
    # Chuẩn hóa khoảng trắng
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def tokenize_input(text):
    """
    Tokenize input text
    """
    try:
        tokens = word_tokenize(text, format="text")
        return tokens.split()
    except:
        return text.split()

def encode_sequence(tokens, word2idx, max_len):
    """
    Encode và pad sequence
    """
    # Encode
    encoded = [word2idx.get(token, word2idx['<UNK>']) for token in tokens]
    
    # Pad hoặc cắt
    if len(encoded) < max_len:
        padded = [word2idx['<PAD>']] * (max_len - len(encoded)) + encoded
    else:
        padded = encoded[-max_len:]
    
    return np.array(padded)

print("✓ Helper functions defined")

In [ ]:
def predict_next_word(input_text, top_k=3, model_type='both'):
    """
    Dự đoán top k từ tiếp theo
    
    Args:
        input_text (str): Văn bản đầu vào
        top_k (int): Số lượng từ dự đoán muốn trả về
        model_type (str): 'ngram', 'lstm', hoặc 'both'
    
    Returns:
        list: Danh sách các từ được dự đoán
    """
    # 1. Tiền xử lý
    cleaned_text = preprocess_input(input_text)
    
    # 2. Tokenize
    tokens = tokenize_input(cleaned_text)
    
    if len(tokens) == 0:
        return []
    
    results = {}
    
    # 3. Dự đoán với N-gram
    if model_type in ['ngram', 'both']:
        ngram_predictions = ngram_model.predict_top_k(tokens, k=top_k)
        results['ngram'] = [word for word, prob in ngram_predictions]
    
    # 4. Dự đoán với LSTM
    if model_type in ['lstm', 'both']:
        # Encode sequence
        encoded_seq = encode_sequence(tokens, word2idx, max_seq_len)
        
        # Reshape cho model
        X_input = encoded_seq.reshape(1, -1)
        
        # Predict
        top_k_indices, top_k_probs = lstm_model.predict_top_k(X_input, k=top_k)
        
        # Decode
        lstm_predictions = [idx2word[idx] for idx in top_k_indices[0]]
        results['lstm'] = lstm_predictions
    
    # 5. Trả về kết quả
    if model_type == 'both':
        return results
    else:
        return results[model_type]

print("✓ predict_next_word() function defined")

## 5. Test với các ví dụ

In [ ]:
# Test case như trong yêu cầu
print("=" * 60)
print("TEST CASE 1: tôi đi học bằng")
print("=" * 60)

input_text = "tôi đi học bằng "
predictions = predict_next_word(input_text, 3, model_type='both')

print(f"\nInput: '{input_text}'\n")
print("N-gram predictions:")
print(predictions['ngram'])
print("\nLSTM predictions:")
print(predictions['lstm'])

In [ ]:
# Nhiều test cases khác
test_cases = [
    "tôi đi học bằng ",
    "hôm nay trời đẹp ",
    "tôi thích ăn ",
    "việt nam là ",
    "chúng tôi đang ",
    "anh ấy là người ",
    "học sinh đi ",
    "cô giáo dạy "
]

print("\n" + "=" * 60)
print("NHIỀU TEST CASES")
print("=" * 60)

for test_input in test_cases:
    print(f"\n{'='*60}")
    print(f"Input: '{test_input}'")
    print(f"{'='*60}")
    
    # N-gram
    ngram_preds = predict_next_word(test_input, 3, model_type='ngram')
    print(f"N-gram: {ngram_preds}")
    
    # LSTM
    lstm_preds = predict_next_word(test_input, 3, model_type='lstm')
    print(f"LSTM:   {lstm_preds}")

## 6. Demo function theo format yêu cầu

In [ ]:
# Function theo đúng format trong yêu cầu
print("=" * 60)
print("DEMO THEO FORMAT YÊU CẦU")
print("=" * 60)

# Sử dụng LSTM làm mặc định
input_example = "tôi đi học bằng "
print(f"\ninput = \"{input_example}\"")
print(f"print(predict_next_word(input, 3))")

result = predict_next_word(input_example, 3, model_type='lstm')
print(result)

print("\n" + "="*60)
print("Các ví dụ khác:")
print("="*60)

examples = [
    ("hôm nay trời ", 3),
    ("tôi thích ", 5),
    ("việt nam ", 3),
]

for inp, k in examples:
    print(f"\ninput = \"{inp}\"")
    print(f"predict_next_word(input, {k}) = {predict_next_word(inp, k, model_type='lstm')}")

## 7. So sánh hiệu suất 2 mô hình

In [ ]:
import time

print("=" * 60)
print("SO SÁNH HIỆU SUẤT")
print("=" * 60)

test_input = "tôi đi học bằng "

# Test N-gram speed
start = time.time()
for _ in range(100):
    _ = predict_next_word(test_input, 3, model_type='ngram')
ngram_time = (time.time() - start) / 100

# Test LSTM speed
start = time.time()
for _ in range(100):
    _ = predict_next_word(test_input, 3, model_type='lstm')
lstm_time = (time.time() - start) / 100

print(f"\nThời gian dự đoán trung bình (100 lần):")
print(f"  N-gram: {ngram_time*1000:.2f} ms")
print(f"  LSTM:   {lstm_time*1000:.2f} ms")
print(f"\nN-gram nhanh hơn: {lstm_time/ngram_time:.2f}x")

## 8. Interactive Testing

In [ ]:
# Interactive test - người dùng có thể nhập input tùy ý
def interactive_test():
    print("=" * 60)
    print("INTERACTIVE TESTING")
    print("=" * 60)
    print("Nhập văn bản để dự đoán từ tiếp theo")
    print("Nhập 'quit' để thoát\n")
    
    while True:
        user_input = input("\nNhập văn bản: ")
        
        if user_input.lower() == 'quit':
            print("Tạm biệt!")
            break
        
        try:
            k = int(input("Số lượng dự đoán (k): "))
        except:
            k = 3
        
        print("\nKết quả:")
        print("-" * 40)
        
        # N-gram
        ngram_result = predict_next_word(user_input, k, model_type='ngram')
        print(f"N-gram: {ngram_result}")
        
        # LSTM
        lstm_result = predict_next_word(user_input, k, model_type='lstm')
        print(f"LSTM:   {lstm_result}")

# Uncomment để chạy interactive mode
# interactive_test()

## 9. Tổng kết

In [ ]:
print("\n" + "=" * 60)
print("TỔNG KẾT")
print("=" * 60)

print("\n✓ Đã load thành công:")
print(f"  - Vocabulary: {vocab_size} từ")
print(f"  - N-gram model: {ngram_model.n}-gram")
print(f"  - LSTM model: {lstm_model.embedding_dim}D embeddings, {lstm_model.hidden_dim}D hidden")

print("\n✓ Các hàm đã tạo:")
print("  - predict_next_word(input_text, top_k, model_type)")
print("    + input_text: Văn bản đầu vào")
print("    + top_k: Số lượng từ dự đoán (mặc định 3)")
print("    + model_type: 'ngram', 'lstm', hoặc 'both' (mặc định 'both')")

print("\n✓ Cách sử dụng:")
print('  input = "tôi đi học bằng "')
print('  predictions = predict_next_word(input, 3, model_type="lstm")')
print('  print(predictions)')

print("\n" + "=" * 60)
print("HOÀN THÀNH!")
print("=" * 60)
print("\nHệ thống dự đoán từ tiếp theo đã sẵn sàng sử dụng!")
print("\nMô hình:")
print("  1. N-gram (Trigram): Dựa trên thống kê n-gram")
print("  2. LSTM: Mạng neural tự xây dựng từ NumPy")
print("\nCả 2 mô hình đều được xây dựng từ đầu không sử dụng")
print("sklearn, PyTorch, hay TensorFlow như yêu cầu!")